In [ ]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/ece1512/projectB/annotated_images.zip .
!unzip annotated_images.zip

Archive:  annotated_images.zip
   creating: annotated_images/
  inflating: annotated_images/.DS_Store  
  inflating: __MACOSX/annotated_images/._.DS_Store  
   creating: annotated_images/test/
   creating: annotated_images/train/
  inflating: annotated_images/test/.DS_Store  
  inflating: __MACOSX/annotated_images/test/._.DS_Store  
   creating: annotated_images/test/HP/
   creating: annotated_images/test/SSA/
  inflating: annotated_images/train/.DS_Store  
  inflating: __MACOSX/annotated_images/train/._.DS_Store  
   creating: annotated_images/train/HP/
   creating: annotated_images/train/SSA/
  inflating: annotated_images/test/HP/MHIST_dkl.png  
  inflating: annotated_images/test/HP/MHIST_dhw.png  
  inflating: annotated_images/test/HP/MHIST_did.png  
  inflating: annotated_images/test/HP/MHIST_auy.png  
  inflating: annotated_images/test/HP/MHIST_ayz.png  
  inflating: annotated_images/test/HP/MHIST_cvw.png  
  inflating: annotated_images/test/HP/MHIST_and.png  
  inflating: annotat

In [ ]:
import torch
from torchvision import transforms as T
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import os
from torchvision import models as models
torch.manual_seed(0)

dataset_path = "annotated_images"
train_transform = T.Compose([
    T.RandomHorizontalFlip(0.5),
    T.RandomVerticalFlip(0.5),
    T.RandomRotation(180),
    T.ToTensor(),
])

test_transform = T.Compose([
    T.ToTensor()
])
    
train_dataset = ImageFolder(os.path.join(dataset_path, "train"), transform=train_transform)
test_dataset = ImageFolder(os.path.join(dataset_path, "test"), transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)




In [ ]:
PATH="/content/drive/MyDrive/Colab Notebooks/ece1512/projectB/MHIST_teacher_layers1234.pt"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = torch.load(PATH, map_location=torch.device('cpu'))

student_model = models.mobilenet_v2(pretrained=True)

for name, param in student_model.named_parameters():
  param.requires_grad = False

lr = 1e-3
new_classifier = torch.nn.Sequential(torch.nn.Dropout(0.2), 
                                      torch.nn.Linear(1280, 2))
student_model.classifier = new_classifier
cross_entropy = torch.nn.CrossEntropyLoss()
nll = torch.nn.NLLLoss()
logsoftmax = torch.nn.LogSoftmax()
softmax = torch.nn.Softmax()
optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
student_model.train()
teacher_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
## transfer learning
def soft_cross_entropy(input, target):
    logprobs = torch.nn.functional.log_softmax (input, dim = 1)
    return  -(target * logprobs).sum() / input.shape[0]

def compute_student_loss(images, label, student, teacher, alpha, temperature):
  student_logits = student(images)
  teacher_logits = teacher(images)

  soft_targets = softmax(teacher_logits / temperature)
  distill_loss = soft_cross_entropy(student_logits / temperature, soft_targets) * temperature ** 2

  loss = (1 - alpha) * cross_entropy(student_logits, label) + alpha * distill_loss
  return loss

student_model.to(device)
teacher_model.to(device)
epochs = 10
alpha = 0.5
temperature = 1
for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = compute_student_loss(inputs, labels, student_model, teacher_model, alpha, temperature)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)
    print("Loss: {}".format(train_loss / len(train_loader.dataset)))
student_model.eval()
correct = 0
for inputs, labels in test_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  pred = torch.argmax(student_model(inputs), axis=1)
  correct += torch.sum(pred == labels)
print("Test Accuracy: {}".format(correct / len(test_loader.dataset)))

Epoch: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Loss: 0.5812308572900706
Epoch: 1
Loss: 0.47942429112291884
Epoch: 2
Loss: 0.458748528327065
Epoch: 3
Loss: 0.4543191594913088
Epoch: 4
Loss: 0.4586168468409571
Epoch: 5
Loss: 0.44629034886414976
Epoch: 6
Loss: 0.43040370720556415
Epoch: 7
Loss: 0.4266630060097267
Epoch: 8
Loss: 0.4472748198728452
Epoch: 9
Loss: 0.4354267371934036
Test Accuracy: 0.7287614941596985


In [ ]:
## fine tuning. Unlock some layers
for name, param in student_model.named_parameters():
  if "features.18" in name or "features.17":
    param.requires_grad = True

lr = 1e-4
optimizer = torch.optim.Adam(student_model.parameters(), lr=lr)
student_model.train()
teacher_model.eval()
student_model.to(device)
teacher_model.to(device)
epochs = 25
for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = compute_student_loss(inputs, labels, student_model, teacher_model, alpha, temperature)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*inputs.size(0)
        
    print("Loss: {}".format(train_loss / len(train_loader.dataset)))
student_model.eval()
correct = 0
for inputs, labels in test_loader:
  inputs, labels = inputs.to(device), labels.to(device)
  pred = torch.argmax(student_model(inputs), axis=1)
  correct += torch.sum(pred == labels)
print("Test Accuracy: {}".format(correct / len(test_loader.dataset)))


Epoch: 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


Loss: 0.43132606895490627
Epoch: 1
Loss: 0.3350223814481976
Epoch: 2
Loss: 0.3462327936874039
Epoch: 3
Loss: 0.2926813584634627
Epoch: 4
Loss: 0.2891384248760925
Epoch: 5
Loss: 0.2787026165545672
Epoch: 6
Loss: 0.25085414443892995
Epoch: 7
Loss: 0.2464979254103255
Epoch: 8
Loss: 0.24009425971699858
Epoch: 9
Loss: 0.23054662938775689
Epoch: 10
Loss: 0.22389420945068886
Epoch: 11
Loss: 0.21119938679810227
Epoch: 12
Loss: 0.21659357092846399
Epoch: 13
Loss: 0.21285928561769682
Epoch: 14
Loss: 0.2065631084024221
Epoch: 15
Loss: 0.19464339500186087
Epoch: 16
Loss: 0.198183763712302
Epoch: 17
Loss: 0.2014589499605113
Epoch: 18
Loss: 0.1819555319725782
Epoch: 19
Loss: 0.17705635805239622
Epoch: 20
Loss: 0.18767752635753018
Epoch: 21
Loss: 0.19036158907002415
Epoch: 22
Loss: 0.16703364536680024
Epoch: 23
Loss: 0.1546147728103331
Epoch: 24


In [ ]:
torch.save(student_model, "/content/drive/MyDrive/Colab Notebooks/ece1512/projectB/MHIST_student_distill_T_{}.pt".format(temperature))